<a href="https://colab.research.google.com/github/xprilion/sasta-perplexity/blob/main/SastaPerplexityQdrantIngest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%%capture
!pip install datasets

In [1]:
%%capture
!pip install qdrant-client[fastembed]

In [2]:
import pandas as pd
import json
import os
import time
from tqdm import tqdm
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import uuid

from google.colab import userdata

In [3]:
collection_name = "search_data"

In [5]:
QDRANT_URL = "https://qdrant-1.sg-1.cloudtop.dev"
QDRANT_KEY = userdata.get('PERSONAL_QDRANT_KEY')

In [6]:
qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_KEY, port=None)

In [7]:
documents = []
metadatas = []
ids = []

In [21]:
from datasets import load_dataset
from tqdm import tqdm

In [12]:
# Load dataset in streaming mode
dataset = load_dataset("google/wit", "train", streaming=True)

In [22]:
data = []

In [23]:
# Access the training split
stream = dataset["train"]

counter = 0
for example in tqdm(stream):
    if example["language"] == "en":
        data.append(example)
        counter += 1
        if counter == 500:
            break

3425it [00:01, 2773.50it/s]


In [24]:
data[0]

{'language': 'en',
 'page_url': 'https://en.wikipedia.org/wiki/Oxydactylus',
 'image_url': 'https://upload.wikimedia.org/wikipedia/commons/5/5f/Oxydactylus_longipes_fm.jpg',
 'page_title': 'Oxydactylus',
 'section_title': None,
 'hierarchical_section_title': 'Oxydactylus',
 'caption_reference_description': None,
 'caption_attribution_description': 'English: Mounted skeleton of Oxydactylus longipes in the Field Museum of Natural History.',
 'caption_alt_text_description': None,
 'mime_type': 'image/jpeg',
 'original_height': 3564,
 'original_width': 2748,
 'is_main_image': True,
 'attribution_passes_lang_id': True,
 'page_changed_recently': True,
 'context_page_description': 'Oxydactylus is an extinct genus of camelid endemic to North America. It lived from the Late Oligocene to the Middle Miocene, existing for approximately 14 million years. The name is from the Ancient Greek οξύς and δάκτυλος.\nThey had very long legs and necks, and were probably adapted to eating high vegetation, muc

In [28]:
for row in tqdm(data):
    page_title = row["page_title"]
    page_description = row["context_page_description"]
    image_caption = row["caption_attribution_description"]
    page_url = row["page_url"]
    image_url = row["image_url"]

    document_text = f"Title: {page_title}\nDescription: {page_description}"
    document_image = f"Title: {page_title}\nDescription: {image_caption}"

    metadata_text = {
        "title": page_title,
        "description": page_description,
        "url": page_url,
        "content_type": "text"
    }

    metadata_img = {
        "title": page_title,
        "description": image_caption,
        "url": image_url,
        "content_type": "image"
    }

    # # Generate a unique ID
    doc_id_text = str(uuid.uuid4())
    doc_id_img = str(uuid.uuid4())

    documents.append(document_text)
    documents.append(document_image)
    metadatas.append(metadata_text)
    metadatas.append(metadata_img)
    ids.append(doc_id_text)
    ids.append(doc_id_img)

100%|██████████| 500/500 [00:00<00:00, 38297.15it/s]


In [29]:
len(documents)

1000

In [30]:
qdrant_client.add(
    collection_name=collection_name,
    documents=documents,
    metadata=metadatas,
    ids=ids
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

['8a658e73-0141-40d6-b1c6-0c26d6f5f85f',
 'f2526a11-00d0-48b7-8bba-73cd78290d82',
 'eaebb0ad-5c1a-4fdb-962d-a61c07d63211',
 'd2ca12c9-d968-4fd4-8a1c-79576049a071',
 'ad68cbee-30ce-4226-b455-8b65ba6bd9e5',
 '43112ccc-ccdc-4d2d-8cd6-866896d72bfc',
 '6632b348-e9d7-40d3-a405-248003b80bda',
 'f9a95c66-3a24-4fee-927a-82f574863895',
 '10d937a8-a1de-470d-be1e-2a86df4a49b7',
 '70978360-2b87-42ae-a26b-58cf28aa31d3',
 '615d5e47-f452-4726-abf9-84f116126bdc',
 'fbe6f264-e954-419d-a2ed-3089b1c317ce',
 '4f72f7e5-d9c3-47f0-9f03-bb193669a5d9',
 '224a2c6e-566a-4e7c-92d4-fb7d4521a3be',
 '081bc30d-e583-4254-bcd4-45ab3854d764',
 '0de8af58-0367-4b95-8d29-973e696b0e76',
 '1da7dc9d-b06a-40d5-8633-423eda68d878',
 '37847047-6ae1-4bd2-94ca-f50a72cfaa40',
 '7ce7689d-1369-45a0-8915-927738f09f8b',
 'd8ce17ee-1e29-4efb-9c6e-df7a19309718',
 '8e2c019f-d253-4744-b249-052b68afd1c0',
 '0edacef1-6dea-4529-aac6-a0e60a919a32',
 'b3de6212-b86f-4047-95ed-4bff8a073534',
 '294e04a8-9665-4f64-b933-7d9e311e4c3f',
 '9531bee6-c9ba-